In [ ]:
import yaml
from pathlib import Path
import zarr
import fly_organelles.utils as utils



In [ ]:
data_config = open("../all_data.yaml", "r")

In [ ]:
def remove_multiscale(zarr_grp, multiscale_name):
    print(f"Removing {multiscale_name} from {zarr_grp.store.path} {zarr_grp.path}")
    multiscales = zarr_grp.attrs["multiscales"]
    multiscales = [ms for ms in multiscales if ms["name"] != multiscale_name]
    zarr_grp.attrs["multiscales"] = multiscales
    zarr_grp.attrs.update(zarr_grp.attrs)

In [ ]:
datas = yaml.safe_load(data_config)
for key, ds_info in datas["datasets"].items():
    print(f"Processing {key}")
    # check raw data to see if it is isotropic
    raw_zarr_grp = zarr.open(
        Path(ds_info["em"]["data"]) / ds_info["em"]["group"], "r+"
    )
    raw_multiscales = utils.get_multiscale_names(raw_zarr_grp)
    if "nominal" in raw_multiscales and "estimated" in raw_multiscales:
        # keep estimated only
        remove_multiscale(raw_zarr_grp, "nominal")
    
        # prepend_multiscale(raw_zarr_grp, ms_nominal.model_dump())
    for crop in ds_info["labels"]["crops"]:
        for cropname in crop.split(","):
            print(f"Processing {cropname}")
            crop_grp = zarr.open(
                Path(ds_info["labels"]["data"])
                / ds_info["labels"]["group"]
                / cropname,
                "r+",
            )
            annotated_classes = set(
                crop_grp.attrs["cellmap"]["annotation"]["class_names"]
            )
            for class_name in sorted(annotated_classes):
                annotated_multiscales = utils.get_multiscale_names(
                    crop_grp[class_name]
                )
                if "nominal" in annotated_multiscales and "estimated" in annotated_multiscales:
                    # keep estimated only
                    remove_multiscale(crop_grp[class_name], "nominal")
                

In [ ]:
datas